# Evolutionary Camouflage Versus a Learning Predator Population

---

TestEvoCamoVsLearnPredPop.ipynb

Started as a copy of EvoCamoVsLearnPredPop.ipynb as of 20220822

On 20220716 starting from here to prototype “local” version running on my Apple Silicon M1 laptop. Got as far as trying to read pre-tained “FCD6_rc4” model but it complained it was compiled for Intel, while this is Apple Silicon.

OK now on 20220722 I am converting this back to "Rube Goldberg" mode where predator vision happens in the cloud, and camouflage evolution runs on my laptop. I tried using the [TensorFlow-Metal plugin](https://developer.apple.com/metal/tensorflow-plugin/), but ran into a [bug](https://developer.apple.com/forums/thread/706920). Until that is fixed I will fall back to the old half-cloud-half-laptop approach.

In [1]:
# "Rube Goldberg" mode refers to running camouflage evolution on my laptop while
# running predator vision in cloud via Colab. State is passed back and forth via
# files on Google Drive.

# TODO 20220822
# Rube_Goldberg_mode = True
Rube_Goldberg_mode = False

def if_RG_mode(for_RG_mode, for_normal_mode):
    return for_RG_mode if Rube_Goldberg_mode else for_normal_mode

# PredatorEye directory on Drive.
pe_directory = '/content/drive/My Drive/PredatorEye/'

# Shared "communication" directory on Drive.
# TODO 20220822
# shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
#                               '/Users/cwr/comms/')
shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
                              '/Users/cwr/camo_data/comms/')

# This was meant (20220716) to allow reading original pre-trained model from
# Google Drive, but I'll need to retrain it for M1 (Apple Silicon).
g_drive_pe_dir = ('/Users/cwr/Library/CloudStorage/' +
                  'GoogleDrive-craig.w.reynolds@gmail.com/' +
                  'My Drive/PredatorEye/')

# Directory for pre-trained Keras/TensorFlow models.
saved_model_directory = if_RG_mode(pe_directory, g_drive_pe_dir) + 'saved_models/'


print('Rube_Goldberg_mode =', Rube_Goldberg_mode)
print('shared_directory =', shared_directory)
print('saved_model_directory =', saved_model_directory)

# Pathname of pre-trained Keras/TensorFlow model
# trained_model = saved_model_directory + '20220202_1211_Find_3_Disks_complex'
# trained_model = saved_model_directory + '20220222_1747_F3D_augmented_rc4'
# trained_model = saved_model_directory + '20220227_0746_F3D2_a'
# trained_model = saved_model_directory + '20220304_1135_FCD5_a'
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'
# model = []

# Directory on Drive for storing fine-tuning dataset.
fine_tuning_directory = shared_directory + 'fine_tuning/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from pathlib import Path

from tensorflow import keras
import numpy as np
import random

# %tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
if Rube_Goldberg_mode:
    sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
else:
    sys.path.append('/Users/cwr/Documents/code/PredatorEye/')
import DiskFind as df

Rube_Goldberg_mode = False
shared_directory = /Users/cwr/camo_data/comms/
saved_model_directory = /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/
TensorFlow version: 2.9.2


# Ad hoc “predator server”

In [2]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        perform_step(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def perform_step(step, directory):
    wait_for_reply(step, shared_directory)
    write_response_file(step, shared_directory)
    delete_find_file(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
def write_response_file(step, directory):
    # Read image file and check for expected format.
    image_pathname = make_camo_pathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    # Default Predator's model -- for prototying.
    model = Predator.default_predator().model
    # Run Predator's model on new image.
    prediction = model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    
        
    ###############################################################################
    # TODO 20220822 move this up before response file write
    # Merge this step's image into fine-tuning dataset, and related bookkeeping.
    fine_tuning_dataset.update(pixel_tensor, prediction, step, directory)
    ###############################################################################

    
    # Generate response file.
    response_string = str(prediction[0]) + " " + str(prediction[1])
    verify_comms_directory_reachable()
    with open(make_find_pathname(step, directory), 'w') as file:
        file.write(response_string)
    print('Wrote ' + "'" + response_string + "'",
          'to response file', Path(make_find_pathname(step, directory)).name)
    # Logging, maybe only for testing, delete me?
    sp = Predator.second_predator()
    spm_predict = sp.model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    print('"other" prediction:', spm_predict,
          ' distance to original prediction:', df.dist2d(spm_predict, prediction))

    ###############################################################################
    # TODO 20220822 move this up before response file write
    #     # Merge this step's image into fine-tuning dataset, and related bookkeeping.
    #     fine_tuning_dataset.update(pixel_tensor, prediction, step, directory)
    ###############################################################################


    # Predator learns from recent experience.
    print('fine-tune default_predator:')
    Predator.default_predator().fine_tune_model(pixel_tensor, prediction,
                                                step, directory)
    # For testing/prototyping
    print('fine-tune second_predator:')
    sp.fine_tune_model(pixel_tensor, prediction, step, directory)

# Delete the given file, usually after having written the next one.
def delete_find_file(step, directory):
    # Why doesn't pathlib provide a Path.remove() method like os?
    # TODO oh, missing_ok was added at pathlib version 3.8.
    # Path(makeMyPathname(step, directory)).unlink(missing_ok=True)
    p = Path(make_find_pathname(step, directory))
    if p.exists():
        p.unlink()

# Delete any remaining file in commuications directory to start a new run.
def clean_up_communication_directory():
    def delete_directory_contents(directory_path):
        for path in directory_path.iterdir():
            print('Removing from communication directory:', path)
            if path.is_dir():
                delete_directory_contents(path)
                path.rmdir()
            else:
                path.unlink()
    delete_directory_contents(Path(shared_directory))

# From pathname for file of given step number from the "other" agent.
def make_camo_pathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for "find_xx.txt" response file from "this" agent.
def make_find_pathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Form pathname for "prey_xx.txt" ground truth file from "other" agent.
def make_prey_pathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'

# Used to ping the comms directory when it seems hung.
def write_ping_file(count, step, directory):
    pn = directory + 'ping_cloud_' + str(step) + '.txt'
    verify_comms_directory_reachable()
    with open(pn, 'w') as file:
        file.write(str(count))
    print('Ping comms: ', count, pn)

# Wait until other agent's file for given step appears.
def wait_for_reply(step, directory):
    camo_pathname = Path(make_camo_pathname(step, directory))
    camo_filename = camo_pathname.name
    prey_pathname = Path(make_prey_pathname(step, directory))
    prey_filename = prey_pathname.name
    print('Waiting for', camo_filename, 'and', prey_filename, '...',
          end='', flush=True)
    start_time = time.time()
    # Loop until both files are present, waiting 1 second between tests.
    test_count = 0
    while not (is_file_present(camo_pathname) and
               is_file_present(prey_pathname)):
        time.sleep(1)
        test_count += 1
        if (test_count % 100) == 0:
            write_ping_file(test_count, step, directory)
    print(' done, elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists()
def is_file_present(file):
    result = False
    verify_comms_directory_reachable()
    filename = Path(file).name
    directory = Path(file).parent
    for i in directory.iterdir():
        if i.name == filename:
            result = True
    return result

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in Path(directory).iterdir():
        name = filename.name
        if other_prefix == name[0:len(other_prefix)]:
            steps.append(int(name.split(".")[0].split("_")[1]))
    return max(steps)

# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    # Read contents of file as string.
    verify_comms_directory_reachable()
    with open(make_prey_pathname(step, directory), 'r') as file:
        prey_centers_string = file.read()
    # Split string at whitespace, map to 6 floats, reshape into 3 xy pairs.
    # (TODO could probably be rewritten cleaner with "list comprehension")
    array = np.reshape(list(map(float, prey_centers_string.split())), (3, 2))
    return array.tolist()

# Keep log of in_disk metric.
def write_in_disk_log(step, history):
    if step % 10 == 0:
        in_disk = history.history["in_disk"][0]
        pathname = shared_directory + 'in_disk_log.csv'
        verify_comms_directory_reachable()
        with open(pathname, 'a') as file:
            if step == 0:
                file.write('step,in_disk\n')
            file.write(str(step) + ',' + "{:.4f}".format(in_disk) + '\n')

# Just wait in retry loop if shared "comms" directory become unreachable.
# Probably will return shortly, better to wait than signal a file error.
# (This is called from places with a local "directory" but it uses global value.)
def verify_comms_directory_reachable():
    seconds = 0
    # shared_directory_pathname = Path(shared_directory)
    # while not shared_directory_pathname.is_dir():
    while not Path(shared_directory).is_dir():
        print("Shared “comms” directory,", shared_directory, 
              "has been inaccessible for", seconds, "seconds.")
        time.sleep(1)  # wait 1 sec
        seconds += 1

# Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
def sort_xy3_by_proximity_to_point(xy3, point):
    # print('xy3 =', xy3)
    xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
    # print('xy3_plus_distance =', xy3_plus_distance)
    sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
    # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
    sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
    # print('sorted_xy3 =', sorted_xy3)
    return sorted_xy3

# Read pre-trained model

As I integrate this into the Predator class, this is no longer “Read pre-trained model” but more like “Some utilities for reading the pre-trained model”

In [3]:
# Read pre-trained TensorFlow "predator vision" model.

# print('Reading pre-trained model from:', trained_model)

# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

def read_default_pre_trained_model():
    print('Reading pre-trained model from:', trained_model)
    return keras.models.load_model(trained_model, custom_objects=dependencies)

# FineTuningDataset

In [4]:
class FineTuningDataset:
    """Manages the dataset of images and labels for fine-tuning."""

    # Accumulated a new “training set” of the most recent N steps seen so far. (See
    # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
    # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
    # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
    max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
    # List of "pixel tensors".
    fine_tune_images = []
    # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
    fine_tune_labels = []

    def update(self, pixel_tensor, prediction, step, directory):
        
        # TODO 20220822 debugging
        time.sleep(5)
        
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # Accumulate the most recent "max_training_set_size" training samples.
        self.fine_tune_images.append(pixel_tensor)
        self.fine_tune_labels.append(sorted_xy3)

        # If training set has become too large, slice off first element of each.
        if len(self.fine_tune_images) > self.max_training_set_size:
            self.fine_tune_images = self.fine_tune_images[1:]
            self.fine_tune_labels = self.fine_tune_labels[1:]

        # print('images_array.shape =', images_array.shape,
        #       '-- labels_array.shape =', labels_array.shape)
        # print('np.shape(self.fine_tune_images) =',
        #       np.shape(self.fine_tune_images),
        #       '-- np.shape(self.fine_tune_labels) =',
        #       np.shape(self.fine_tune_labels))
        print('fine_tune_images shape =', np.shape(self.fine_tune_images),
              '-- fine_tune_labels shape =', np.shape(self.fine_tune_labels))
        

# Create a global FineTuningDataset object.
# (TODO globals are usually a bad idea, reconsider this.)
fine_tuning_dataset = FineTuningDataset()

# Predator class

In [5]:
class Predator:
    """Represents a Predator in the camouflage simulation. It has a CNN-based
       model of visual hunting that identified the position of likely prey."""

    # Global list of active Predators. (As a class variable.)
    population = []

    def __init__(self):
        # Each Predator has its own copy of a prey-finding CNN-based model.
        self.model = []
        # By default add this new Predator to the population (TODO reconsider)
        Predator.population.append(self)

    # Set this Predator's model to default.
    def initialize_to_pre_trained_model(self):
        self.model = read_default_pre_trained_model()

    # This are presumably just for prototyping
    def default_predator():
        return Predator.population[0]
    def second_predator():
        return Predator.population[1]
    
    # Keep track of how often selected prey is nearest center:
    nearest_center = 0

    # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
    # training set. Assume they were "near misses" and so training label is actual
    # (ground truth) center of disk nearest prediction. Keep a max number of old
    # steps to allow gradually forgetting the earliest part of the run.
    def fine_tune_model(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # keep track of how often selected prey is nearest center:
        temp = xy3.copy()  # needed?
        sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
        if sorted_by_dist_to_center[0] == sorted_xy3[0]:
            Predator.nearest_center += 1
        print('  nearest_center:',
              str(int(100 * float(self.nearest_center) / (step + 1))) + '%',
              '(nearest_center =', self.nearest_center, ', steps =', step + 1, ')')

        # Convert training data list to np arrays
        images_array = np.array(fine_tuning_dataset.fine_tune_images)
        labels_array = np.array([x[0] for x in fine_tuning_dataset.fine_tune_labels])

        # print('images_array.shape =', images_array.shape,
        #       '-- labels_array.shape =', labels_array.shape)

    	# Skip fine-tuning until dataset is large enough (10% of max size).
        ########################################################################
        # TODO 20220803 clone model

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # TODO 20220823 restore "Skip fine-tuning until dataset is large enough"
        if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
            # # Do fine-tuning training step using data accumulated during run.
            # history = self.model.fit(x=images_array, y=labels_array)
            
            # TODO 20220823 -- run fine-tuning on CPU only.
            print('Running on CPU ONLY!')
            with tf.device('/cpu:0'):
                # Do fine-tuning training step using data accumulated during run.
                history = self.model.fit(x=images_array, y=labels_array)
            
            # Keep log of in_disk metric:
            write_in_disk_log(step, history)


        ########################################################################
    
    ############################################################################
    # TODO 20220803 clone model

    # # Copy the neural net model of a given predator into this one.
    # def copy_model(self, another_predator):
    #     # No this is wrong, just does a shallow copy
    #     # self.model = another_predator.model
    #     self.model = tf.keras.models.clone_model(another_predator.model)
    #     # Compile newly cloned model.
    #     # compile_disk_finder_model(self.model)
    #     df.compile_disk_finder_model(self.model)

    # Copy the neural net model of a given predator into this one. (From "Make
    # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
    def copy_model(self, another_predator):
        other_model = another_predator.model
        # Clone layer structure of other model.
        # self.model = tf.keras.models.clone_model(another_predator.model)
        self.model = tf.keras.models.clone_model(other_model)
        # Compile newly cloned model.
        df.compile_disk_finder_model(self.model)

        # Copy weights of other model.
        self.model.set_weights(other_model.get_weights())

    ############################################################################

    # Modify this Predator's model by adding noise to its weights.
    def jiggle_model(self):
        weight_perturbation(self.model, 0.001)


# Utility based on https://stackoverflow.com/a/64542651/1991373
def weight_perturbation(model, max_range):
    """Add noise to all weights in a Keras model."""
    for layer in model.layers:
        trainable_weights = layer.trainable_variables
        for weight in trainable_weights:
            random_weights = tf.random.uniform(tf.shape(weight),
                                               max_range / -2,
                                               max_range / 2,
                                               dtype=tf.float32)
            weight.assign_add(random_weights)

############################################################################
# TODO 20220803 clone model
#     This should be a utility in DiskFind.py,
#     and be called from make_disk_finder_model()

# # Compile a disk finder model.
# def compile_disk_finder_model(model):
#     # Compile with mse loss, tracking accuracy and fraction-inside-disk.
#     model.compile(loss='mse', optimizer='adam', metrics=["accuracy", df.in_disk])

# # Compile a disk finder model.
# def compile_disk_finder_model(model):
#     # Compile with mse loss, tracking accuracy and fraction-inside-disk.
#     model.compile(loss='mse', optimizer='adam', metrics=["accuracy", in_disk])

############################################################################


# # Global Predator instances for prototyping.
# test_predator = Predator()
# test_predator.initialize_to_pre_trained_model()
# second_predator = Predator()
# # second_predator.initialize_to_pre_trained_model()
# second_predator.copy_model(test_predator)
# second_predator.jiggle_model()

# Global Predator instances for prototyping.
test_predator = Predator()
test_predator.initialize_to_pre_trained_model()
second_predator = Predator()
second_predator.copy_model(test_predator)
# TODO leave second_predator an exact copy of first_predator to test equivalence
#second_predator.jiggle_model()

Reading pre-trained model from: /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4


# Run test

In [ ]:
# Keep track of how often selected prey is nearest center:
Predator.nearest_center = 0

# Predator.population = []
# TODO maybe a Predator.reset() method?

# Flush out obsolete files in comms directory.
clean_up_communication_directory()

# Start fresh run defaulting to step 0.
start_run()

Removing from communication directory: /Users/cwr/camo_data/comms/ping_earth_754.txt
Removing from communication directory: /Users/cwr/camo_data/comms/find_752.txt
Removing from communication directory: /Users/cwr/camo_data/comms/find_753.txt
Removing from communication directory: /Users/cwr/camo_data/comms/in_disk_log.csv
Removing from communication directory: /Users/cwr/camo_data/comms/camo_754.png
Removing from communication directory: /Users/cwr/camo_data/comms/prey_754.txt
Removing from communication directory: /Users/cwr/camo_data/comms/prey_753.txt
Start run in /Users/cwr/camo_data/comms/
Waiting for camo_0.png and prey_0.txt ... done, elapsed time: 10 seconds.
1/1 [==============================] - 0s 96ms/step
fine_tune_images shape = (1, 128, 128, 3) -- fine_tune_labels shape = (1, 3, 2)
Wrote '0.41181424 0.22343275' to response file find_0.txt
1/1 [==============================] - 0s 66ms/step
"other" prediction: [0.41181424 0.22343275]  distance to original prediction: 0.0

1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.53175   0.5775293]  distance to original prediction: 0.0
fine-tune default_predator:
  nearest_center: 92% (nearest_center = 13 , steps = 14 )
fine-tune second_predator:
  nearest_center: 100% (nearest_center = 14 , steps = 14 )
Waiting for camo_14.png and prey_14.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/step
fine_tune_images shape = (15, 128, 128, 3) -- fine_tune_labels shape = (15, 3, 2)
Wrote '0.30707207 0.6538952' to response file find_14.txt
1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.30707207 0.6538952 ]  distance to original prediction: 0.0
fine-tune default_predator:
  nearest_center: 93% (nearest_center = 14 , steps = 15 )
fine-tune second_predator:
  nearest_center: 93% (nearest_center = 14 , steps = 15 )
Waiting for camo_15.png and prey_15.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 10ms/st

Waiting for camo_28.png and prey_28.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/step
fine_tune_images shape = (29, 128, 128, 3) -- fine_tune_labels shape = (29, 3, 2)
Wrote '0.42963314 0.8098514' to response file find_28.txt
1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.42963314 0.8098514 ]  distance to original prediction: 0.0
fine-tune default_predator:
  nearest_center: 117% (nearest_center = 34 , steps = 29 )
fine-tune second_predator:
  nearest_center: 117% (nearest_center = 34 , steps = 29 )
Waiting for camo_29.png and prey_29.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 10ms/step
fine_tune_images shape = (30, 128, 128, 3) -- fine_tune_labels shape = (30, 3, 2)
Wrote '0.55812323 0.7764269' to response file find_29.txt
1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.55812323 0.7764269 ]  distance to original prediction: 0.0
fine-tune default_predato

1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.22160497 0.5393406 ]  distance to original prediction: 0.0
fine-tune default_predator:
  nearest_center: 118% (nearest_center = 51 , steps = 43 )
fine-tune second_predator:
  nearest_center: 120% (nearest_center = 52 , steps = 43 )
Waiting for camo_43.png and prey_43.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/step
fine_tune_images shape = (44, 128, 128, 3) -- fine_tune_labels shape = (44, 3, 2)
Wrote '0.41983283 0.637' to response file find_43.txt
1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.41983283 0.637     ]  distance to original prediction: 0.0
fine-tune default_predator:
  nearest_center: 118% (nearest_center = 52 , steps = 44 )
fine-tune second_predator:
  nearest_center: 118% (nearest_center = 52 , steps = 44 )
Waiting for camo_44.png and prey_44.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/s

2/2 [==============================] - 0s 134ms/step - loss: 0.0154 - accuracy: 0.8000 - in_disk: 0.3636
fine-tune second_predator:
  nearest_center: 101% (nearest_center = 56 , steps = 55 )
Running on CPU ONLY!
2/2 [==============================] - 0s 135ms/step - loss: 0.0201 - accuracy: 0.7636 - in_disk: 0.3636
Waiting for camo_55.png and prey_55.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 11ms/step
fine_tune_images shape = (56, 128, 128, 3) -- fine_tune_labels shape = (56, 3, 2)
Wrote '0.6762245 0.5199648' to response file find_55.txt
1/1 [==============================] - 0s 9ms/step
"other" prediction: [0.6745657 0.5120791]  distance to original prediction: 0.008058286
fine-tune default_predator:
  nearest_center: 101% (nearest_center = 57 , steps = 56 )
Running on CPU ONLY!
2/2 [==============================] - 0s 140ms/step - loss: 0.0220 - accuracy: 0.7679 - in_disk: 0.4821
fine-tune second_predator:
  nearest_center: 103% (nearest_center

...

16/16 [==============================] - 3s 194ms/step - loss: 0.0029 - accuracy: 0.9120 - in_disk: 0.8920
Waiting for camo_1588.png and prey_1588.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 8ms/step
fine_tune_images shape = (500, 128, 128, 3) -- fine_tune_labels shape = (500, 3, 2)
Wrote '0.52031726 0.24269156' to response file find_1588.txt
1/1 [==============================] - 0s 9ms/step
"other" prediction: [0.36147115 0.40754044]  distance to original prediction: 0.22892627
fine-tune default_predator:
  nearest_center: 75% (nearest_center = 1204 , steps = 1589 )
Running on CPU ONLY!
16/16 [==============================] - 3s 194ms/step - loss: 0.0028 - accuracy: 0.9340 - in_disk: 0.9080
fine-tune second_predator:
  nearest_center: 75% (nearest_center = 1204 , steps = 1589 )
Running on CPU ONLY!
16/16 [==============================] - 3s 191ms/step - loss: 0.0027 - accuracy: 0.9280 - in_disk: 0.9200
Waiting for camo_1589.png and prey_1589.txt

16/16 [==============================] - 3s 197ms/step - loss: 0.0025 - accuracy: 0.9300 - in_disk: 0.9240
fine-tune second_predator:
  nearest_center: 75% (nearest_center = 1212 , steps = 1598 )
Running on CPU ONLY!
16/16 [==============================] - 3s 202ms/step - loss: 0.0036 - accuracy: 0.9340 - in_disk: 0.8920
Waiting for camo_1598.png and prey_1598.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
fine_tune_images shape = (500, 128, 128, 3) -- fine_tune_labels shape = (500, 3, 2)
Wrote '0.27842888 0.35228747' to response file find_1598.txt
1/1 [==============================] - 0s 12ms/step
"other" prediction: [0.2906815  0.34661725]  distance to original prediction: 0.013501048
fine-tune default_predator:
  nearest_center: 75% (nearest_center = 1212 , steps = 1599 )
Running on CPU ONLY!
16/16 [==============================] - 3s 198ms/step - loss: 0.0043 - accuracy: 0.9080 - in_disk: 0.9020
fine-tune second_predator:
  nearest_cent

1/1 [==============================] - 0s 9ms/step
"other" prediction: [0.6740386  0.25000694]  distance to original prediction: 0.02941504
fine-tune default_predator:
  nearest_center: 75% (nearest_center = 1220 , steps = 1608 )
Running on CPU ONLY!
16/16 [==============================] - 3s 195ms/step - loss: 0.0021 - accuracy: 0.9380 - in_disk: 0.9320
fine-tune second_predator:
  nearest_center: 75% (nearest_center = 1220 , steps = 1608 )
Running on CPU ONLY!
16/16 [==============================] - 3s 194ms/step - loss: 0.0023 - accuracy: 0.9560 - in_disk: 0.9300
Waiting for camo_1608.png and prey_1608.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 8ms/step
fine_tune_images shape = (500, 128, 128, 3) -- fine_tune_labels shape = (500, 3, 2)
Wrote '0.3845213 0.33790398' to response file find_1608.txt
1/1 [==============================] - 0s 10ms/step
"other" prediction: [0.45985925 0.30903363]  distance to original prediction: 0.08068025
fine-tune 

16/16 [==============================] - 3s 203ms/step - loss: 0.0031 - accuracy: 0.9320 - in_disk: 0.8960
Waiting for camo_1617.png and prey_1617.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
fine_tune_images shape = (500, 128, 128, 3) -- fine_tune_labels shape = (500, 3, 2)
Wrote '0.63304424 0.34270656' to response file find_1617.txt
1/1 [==============================] - 0s 11ms/step
"other" prediction: [0.49037087 0.31808114]  distance to original prediction: 0.14478296
fine-tune default_predator:
  nearest_center: 75% (nearest_center = 1228 , steps = 1618 )
Running on CPU ONLY!
16/16 [==============================] - 3s 203ms/step - loss: 0.0032 - accuracy: 0.9300 - in_disk: 0.9120
fine-tune second_predator:
  nearest_center: 75% (nearest_center = 1228 , steps = 1618 )
Running on CPU ONLY!
16/16 [==============================] - 3s 203ms/step - loss: 0.0031 - accuracy: 0.9360 - in_disk: 0.9000
Waiting for camo_1618.png and prey_1618.tx

16/16 [==============================] - 3s 194ms/step - loss: 0.0034 - accuracy: 0.9160 - in_disk: 0.9060
fine-tune second_predator:
  nearest_center: 75% (nearest_center = 1232 , steps = 1627 )
Running on CPU ONLY!
16/16 [==============================] - 3s 190ms/step - loss: 0.0033 - accuracy: 0.9120 - in_disk: 0.8800
Waiting for camo_1627.png and prey_1627.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
fine_tune_images shape = (500, 128, 128, 3) -- fine_tune_labels shape = (500, 3, 2)
Wrote '0.43718702 0.7786623' to response file find_1627.txt
1/1 [==============================] - 0s 9ms/step
"other" prediction: [0.47073382 0.7573892 ]  distance to original prediction: 0.039723225
fine-tune default_predator:
  nearest_center: 75% (nearest_center = 1232 , steps = 1628 )
Running on CPU ONLY!
16/16 [==============================] - 3s 189ms/step - loss: 0.0023 - accuracy: 0.9400 - in_disk: 0.9440
fine-tune second_predator:
  nearest_center

In [ ]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.

# restart_run()